In [2]:
import numpy as np
from keras.callbacks import CSVLogger
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, GRU
import pandas as pd
import os
import random

In [11]:
df = pd.read_excel('output.xlsx', index_col=0)

vecs_ictal = np.load('ictal_feature.npy')
vecs_interictal = np.load('interictal_feature.npy')

person = np.sort(df["patient"].to_numpy())

In [ ]:
def data_splitter(train_data = [1,2,3,4,5,6,7,8,9,10], validation_data=[11,12], test_data=[13]):
    X_train, y_train, X_validation,y_validation, X_test, y_test = [], [], [], [], [], []

    for e, per in enumerate(person):
        if per+1 in train_data:
            X_train.append(all_vecs[per,int(times[e][0]*30):int(times[e][1]*30),:])
            y_train.append(labels[e])

        if per+1 in validation_data:
            X_validation.append(all_vecs[per,int(times[e][0]*30):int(times[e][1]*30),:])
            y_validation.append(labels[e])

        if per+1 in test_data:
            X_test.append(all_vecs[per,int(times[e][0]*30):int(times[e][1]*30),:])
            y_test.append(labels[e])
    return np.array(X_train), np.array(y_train), np.array(X_validation), np.array(y_validation), np.array(X_test), np.array(y_test)